<a href="https://colab.research.google.com/github/abdulwasaeee/Support-Vector-Machines/blob/main/SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score,mean_absolute_error
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [3]:
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
l1=LabelEncoder()
l2=LabelEncoder()
l3=LabelEncoder()
df['sex']=l1.fit_transform(df['sex'])
df['smoker']=l2.fit_transform(df['smoker'])
df['time']=l3.fit_transform(df['time'])
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,0,0,Sun,0,2
1,10.34,1.66,1,0,Sun,0,3
2,21.01,3.50,1,0,Sun,0,3
3,23.68,3.31,1,0,Sun,0,2
4,24.59,3.61,0,0,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,1,0,Sat,0,3
240,27.18,2.00,0,1,Sat,0,2
241,22.67,2.00,1,1,Sat,0,2
242,17.82,1.75,1,0,Sat,0,2


In [5]:
ohe=OneHotEncoder()
daycols=ohe.fit_transform(df[['day']]).toarray()
daycols=pd.DataFrame(daycols,columns=ohe.get_feature_names_out(['day']))
daycols

,day_Fri,day_Sat,day_Sun,day_Thur
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
239,0.0,1.0,0.0,0.0
240,0.0,1.0,0.0,0.0
241,0.0,1.0,0.0,0.0
242,0.0,1.0,0.0,0.0


In [6]:
df=pd.concat([df,daycols],axis=1)
df

,total_bill,tip,sex,smoker,day,time,size,day_Fri,day_Sat,day_Sun,day_Thur
0,16.99,1.01,0,0,Sun,0,2,0.0,0.0,1.0,0.0
1,10.34,1.66,1,0,Sun,0,3,0.0,0.0,1.0,0.0
2,21.01,3.50,1,0,Sun,0,3,0.0,0.0,1.0,0.0
3,23.68,3.31,1,0,Sun,0,2,0.0,0.0,1.0,0.0
4,24.59,3.61,0,0,Sun,0,4,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,1,0,Sat,0,3,0.0,1.0,0.0,0.0
240,27.18,2.00,0,1,Sat,0,2,0.0,1.0,0.0,0.0
241,22.67,2.00,1,1,Sat,0,2,0.0,1.0,0.0,0.0
242,17.82,1.75,1,0,Sat,0,2,0.0,1.0,0.0,0.0


In [7]:
y=df.total_bill
x=df.drop(columns=['total_bill','day'])
x

,tip,sex,smoker,time,size,day_Fri,day_Sat,day_Sun,day_Thur
0,1.01,0,0,0,2,0.0,0.0,1.0,0.0
1,1.66,1,0,0,3,0.0,0.0,1.0,0.0
2,3.50,1,0,0,3,0.0,0.0,1.0,0.0
3,3.31,1,0,0,2,0.0,0.0,1.0,0.0
4,3.61,0,0,0,4,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
239,5.92,1,0,0,3,0.0,1.0,0.0,0.0
240,2.00,0,1,0,2,0.0,1.0,0.0,0.0
241,2.00,1,1,0,2,0.0,1.0,0.0,0.0
242,1.75,1,0,0,2,0.0,1.0,0.0,0.0


In [8]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)


In [9]:
svr=SVR()
svr.fit(x_train,y_train)
y_pred=svr.predict(x_test)

In [10]:
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.4974038696417853
4.457901629454618


In [23]:
params={ 'C': [0.1,1,10,100,1000], 'gamma':[1,0.1,0.01,0.001,0.0001], 'kernel':['rbf'] }
grid=GridSearchCV(estimator=SVR(),param_grid=params,refit=True,verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)
grid.best_params_

Fitting 5 folds for each of 25 candidates, totalling 125 fits


{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [19]:
y_pred2=grid.predict(x_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.4974038696417853
4.457901629454618
